In [262]:
import numpy as np
import matplotlib.pyplot as plt

In [263]:
def next_matrix(Q, w0, x0, w, x, max_iter):
    N = len(x)
    Q_tilde = np.zeros((N+1, N+1))
    Q_tilde[1:, 1:] = Q
    Q_tilde[N, 0] = w0
    
    x_tilde = np.zeros(N+1)
    x_tilde[0] = x0
    for i in range(N):
        x_tilde[i+1] = x[i]

    it = 0
    while(it < max_iter):
        is_compatible = True
        q = np.random.randn(N)

        Q_tilde[0, 0] = - np.sum(q)
        for i in range(N):
            Q_tilde[0, i+1] = q[i]

        # vérification de la compatibilité
        Q_tilde_inv = np.linalg.inv(Q_tilde)
        
        # [Q_tilde Diag(x0,x1,...,xN) Q_tilde-1]ij <= 0 pour i != j
        D_tilde = np.diag(x_tilde)
        A = Q_tilde @ D_tilde @ Q_tilde_inv
        for i in range(N+1):
            for j in range(N+1):
                if(i != j and A[i, j] > 0):
                    is_compatible = False
        it += 1
        
        if(is_compatible):
            return Q_tilde
    return None

In [264]:
Q = np.array([[1, -1],
              [1, 2]])

x = np.array([1, 10])
w = np.array([1, 2])
x0 = 0.5
w0 = 0.3

In [265]:
def compute_A(Q, w0, x0, x, q):
    N = len(x)
    
    # Construction de Q_tilde (matrice (N+1)x(N+1))
    Q_tilde = np.zeros((N+1, N+1))
    Q_tilde[1:, 1:] = Q
    Q_tilde[0, 0] = -np.sum(q)
    Q_tilde[0, 1:] = q
    Q_tilde[N, 0] = w0  # note : w n'est pas utilisé ici

    # Construction de x_tilde (vecteur N+1)
    x_tilde = np.zeros(N+1)
    x_tilde[0] = x0
    x_tilde[1:] = x

    try:
        Q_tilde_inv = np.linalg.inv(Q_tilde)
    except np.linalg.LinAlgError:
        raise ValueError("Q_tilde is not invertible.")

    # Calcul de A = Q_tilde @ diag(x_tilde) @ Q_tilde_inv
    D_tilde = np.diag(x_tilde)
    A = Q_tilde @ D_tilde @ Q_tilde_inv

    return A

def phi(q, Q, w0, x0, w, x, penalty=1e6):
    A = compute_A(Q, w0, x0, x, q)
    
    N = A.shape[0]
    total = 0.0
    for i in range(N):
        for j in range(N):
            if i != j:
                if A[i, j] > 0:
                    total += 50 * A[i, j]
                else:
                    total += A[i, j]
    return total

from scipy.optimize import minimize

import numpy as np
from scipy.optimize import minimize

def optimize_q_sum1(Q, w0, x0, w, x, penalty=1e6):
    N = len(x)
    
    def objective(q):
        return phi(q, Q, w0, x0, w, x, penalty)
    
    # Contrainte : somme des q égale 1
    cons = {
        'type': 'eq',
        'fun': lambda q: np.sum(q) - 1
    }
    
    q0 = np.random.randn(N)
    q0 = q0 / np.sum(q0)  # normaliser q0 pour respecter la contrainte au départ
    
    res = minimize(
        objective,
        x0=q0,
        method='SLSQP',  # méthode supportant contraintes
        constraints=cons,
        options={'disp': True}
    )
    
    if res.success:
        print("Optimisation réussie")
    else:
        print("Échec de l'optimisation :", res.message)
    
    return res.x, res.fun


q_opt, phi_val = optimize_q_sum1(Q, w0, x0, w, x)
print("q optimal :", q_opt)
print("phi(q_opt) :", phi_val)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 96.11712515757011
            Iterations: 12
            Function evaluations: 68
            Gradient evaluations: 12
Optimisation réussie
q optimal : [0.9230308 0.0769692]
phi(q_opt) : 96.11712515757011


In [266]:
print(next_matrix(Q, w0, x0, w, x, 10000))

None
